In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
# os.environ

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)
llm

ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, google_api_key=SecretStr('**********'), model='gemini-2.5-flash-lite', temperature=0.0, client=<google.genai.client.Client object at 0x10c331910>, default_metadata=(), model_kwargs={})

In [8]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content='I\'m sorry, but I don\'t have access to real-time weather information. My knowledge is based on the data I was trained on, which is not current.\n\nTo get the most accurate and up-to-date weather forecast for Munich today, I recommend checking a reliable weather source like:\n\n*   **Weather Websites:** AccuWeather, Weather.com, BBC Weather, etc.\n*   **Weather Apps:** Most smartphones come with a built-in weather app or you can download one.\n*   **Google Search:** Simply search "weather in Munich" and you\'ll get current conditions and a forecast.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68b9-1814-73e0-baaa-7017e374d59e-0', usage_metadata={'input_tokens': 11, 'output_tokens': 131, 'total_tokens': 142, 'input_token_details': {'cache_read': 0}})

In [10]:
from langchain_core.tools import tool

@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"


tools = [fake_weather_api]
tools

[StructuredTool(name='fake_weather_api', description='Check the weather in a specified city.\n\nArgs:\n    city (str): The name of the city where you want to check the weather.\n\nReturns:\n    str: A description of the current weather in the specified city.', args_schema=<class 'langchain_core.utils.pydantic.fake_weather_api'>, func=<function fake_weather_api at 0x10d5fa200>)]

In [11]:

llm_with_tools = llm.bind_tools(tools)
llm_with_tools

RunnableBinding(bound=ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, google_api_key=SecretStr('**********'), model='gemini-2.5-flash-lite', temperature=0.0, client=<google.genai.client.Client object at 0x10c331910>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'fake_weather_api', 'description': 'Check the weather in a specified city.\n\nArgs:\n    city (str): The name of the city where you want to check the weather.\n\nReturns:\n    str: A description of the current weather in the specified city.', 'parameters': {'properties': {'city': {'type': 'string'}}, 'required': ['city'], 'type': 'o

In [12]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'fake_weather_api', 'arguments': '{"city": "Munich"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68bc-2261-7510-a8d9-560164160f32-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'cbb301e6-9abd-4cf5-830b-435b19964de8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 18, 'total_tokens': 113, 'input_token_details': {'cache_read': 0}})

In [13]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'Munich'},
  'id': 'cbb301e6-9abd-4cf5-830b-435b19964de8',
  'type': 'tool_call'}]

In [14]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [15]:
messages

[HumanMessage(content='How will the weather be in munich today?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'fake_weather_api', 'arguments': '{"city": "munich"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68bd-9d79-7111-b7b9-ca32aeddc146-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'munich'}, 'id': '0503f339-77be-4298-aab9-f8e20d25eac3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 18, 'total_tokens': 113, 'input_token_details': {'cache_read': 0}})]

In [16]:
llm_output

AIMessage(content='', additional_kwargs={'function_call': {'name': 'fake_weather_api', 'arguments': '{"city": "munich"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68bd-9d79-7111-b7b9-ca32aeddc146-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'munich'}, 'id': '0503f339-77be-4298-aab9-f8e20d25eac3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 18, 'total_tokens': 113, 'input_token_details': {'cache_read': 0}})

In [17]:
tool_mapping = {"fake_weather_api": fake_weather_api}

In [18]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [19]:
messages

[HumanMessage(content='How will the weather be in munich today?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'fake_weather_api', 'arguments': '{"city": "munich"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68bd-9d79-7111-b7b9-ca32aeddc146-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'munich'}, 'id': '0503f339-77be-4298-aab9-f8e20d25eac3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 18, 'total_tokens': 113, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='Sunny, 22°C', tool_call_id='0503f339-77be-4298-aab9-f8e20d25eac3')]

In [20]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today will be Sunny with a temperature of 22°C.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68be-b5b2-7111-8dc6-c77426894eda-0', usage_metadata={'input_tokens': 136, 'output_tokens': 18, 'total_tokens': 154, 'input_token_details': {'cache_read': 0}})